# How to run workloop experiments using the multifilament model with titin

First we need to check out/download the model's repo from github and switch over to the titin branch

We executed the following:
```
git clone https://github.com/cdw/multifil.git;
cd multifil;
git checkout titin
```

In [ ]:
#Check that we are up to date
%cd ../../multifil/
!git pull
!git status

# Normally I'd encourage installation directly, but we want to override 
# my existing installation of multifil so we need to ensure it is sourced first
import sys
sys.path.insert(0, 'multifil/')
import multifil
import multifil.aws.metas

%cd ../clientMultifil/

In [ ]:
#rest of imports 
import ujson as json
import matplotlib.pyplot as plt

Now we've got the multifil model loaded, working on the titin branch. Let's instantiate a half-sarcomere and run it to genereate some forces. 

In [ ]:
hs = multifil.hs.hs()
forces = hs.run(10)

And plot them

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(forces)
ax.set(xlabel="Timestep (ms)", ylabel="Force (pN)")
plt.tight_layout()

That is a simple (very short) run at maximal activation with default lattice spacing and length. Good for some things, but we want to run workloops. 

The file `multifil.aws.metas` contains various functions intended to generate run files (metas) for different runs. These meta files need traces for lenght and activation if we don't want them to be constant (lattice spacing is currently set by setting poission ratio). Let's look at the traces we need to create for a workloop run:

In [ ]:
time_trace = multifil.aws.metas.time(
    .5, # ms per timestep
    200, # ms to run for
                                    )
length_trace = multifil.aws.metas.zline_workloop(
    1200, # resting hs length 
    10, # peak to peak amp 
    25, # cycle freq in Hz 
    time_trace
                                                )

'''init params freq=25, phase=0.1, stim_duration=20, influx_time=2, half_life=10, time=time'''
'''first twitch: 5, 10, 50, 2, 10'''
'''positive workloop params: 25, # freq in Hz, 0.1, # phase offset, 10, # stimulus duration in ms
    2, # time it takes for ca to go from 10 to 90% of influx level, 10, # half life of Ca decay, time_trace'''
actin_permissiveness_trace = multifil.aws.metas.actin_permissiveness_workloop(
    25, # freq in Hz 
    0.1, # phase offset 
    20, # stimulus duration in ms
    2, # time it takes for ca to go from 10 to 90% of influx level
    10, # half life of Ca decay 
    time_trace
                                                                             )

It is hard to understand those without plotting them together:

In [ ]:
fig, axes = plt.subplots(2,1,sharex=True)
axes[0].plot(time_trace, length_trace)
axes[0].set(ylabel='hs length (nm)')
axes[1].plot(time_trace, actin_permissiveness_trace)
axes[1].set(ylabel='actin permissiveness',
           xlabel='time (ms)')
plt.tight_layout()

Ok, now that we have our wl parameter traces, we can encode them into the meta:

In [ ]:
wl_meta, run_name = multifil.aws.metas.emit('../outputClientMultifil/AAAAAAH/', # local dir to save run output to
                                  None, #s3 bucket to upload results to
                                  time_trace, 
                                  0.5, # poisson ratio, set to const vol here
                                 None, # intial ls, using default by passing none
                                 length_trace, 
                                 actin_permissiveness_trace, 
                                 "an example run", # comment describing run
                                 True, # whether to write out resulting file 
                                  #or just pass back to variable
                                 )
!dir

 We created a meta file locally at  `f07114fe-91c8-11e8-a3d9-2a0001d4a520.meta.json`.
 Actually called `bbf390e6-7696-11e9-a72f-c3083097ab67.meta.json`.
 Let's run it locally too:

In [ ]:
run = multifil.aws.run.manage(run_name, False)

In [ ]:
run.run_and_save()

We can load in the saved data from the run:

In [ ]:
print(run_name)
data_name = run_name.split('.meta')[0] + '.data.json'

with open(data_name, 'r') as file:
    data = json.load(file)

And plot using helper functions in `work_and_workloops`

In [ ]:
import originalWorkAndWorkloops as work_and_workloops
import work_and_workloops

fig, ax = plt.subplots(1,1)
work_and_workloops.workloop_plot(ax, data['z_line'], data['axial_force'])

That was mostly positive work, nice! Those fill-in line mismatches get a whole lot better as the time resolution of the trial gets better.

This concluded the minimal example of how to run a workloop trial. 